In [0]:
import tensorflow as tf
import numpy as np
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
import io

Upload files

In [0]:
from google.colab import files

In [5]:
uploaded = files.upload()

Saving BBC News Sample Solution.csv to BBC News Sample Solution.csv
Saving BBC News Test.csv to BBC News Test.csv
Saving BBC News Train.csv to BBC News Train.csv


Do preprocessing news data text using gensim

In [0]:
df = pd.read_csv(io.StringIO(uploaded['BBC News Train.csv'].decode('utf-8')))

In [0]:
df['Text'] = df['Text'].map(lambda x : preprocess_string(x))

In [29]:
# preprocessed news data( Text => preprocessed Word)
print(df['Text'])

0       [worldcom, boss, launch, defenc, lawyer, defen...
1       [german, busi, confid, slide, german, busi, co...
2       [bbc, poll, indic, econom, gloom, citizen, maj...
3       [lifestyl, govern, mobil, choic, faster, bette...
4       [enron, boss, payout, eighteen, enron, directo...
5       [howard, truant, plai, snooker, conserv, leade...
6       [wale, silent, grand, slam, talk, rhy, william...
7       [french, honour, director, parker, british, fi...
8       [car, giant, hit, merced, slump, slump, profit...
9       [focker, fuel, festiv, film, chart, comedi, me...
10      [blair, reject, iraq, advic, call, toni, blair...
11      [housew, lift, channel, rate, debut, televis, ...
12      [coal, plung, deeper, loss, share, coal, falle...
13      [surg, ahead, high, oil, price, oil, giant, an...
14      [ireland, argentina, injuri, time, drop, goal,...
15      [wenger, sign, new, deal, arsen, manag, arsen,...
16      [desir, number, won, prestigi, grammi, award, ...
17      [hantu